Esse programa cria uma série temporal de inundação

## 1. São importadas as bibliotecas


In [1]:
import ee
import datetime as dt
import pandas as pd
import fiona

## Agora, se inicializa o Earth Engine

In [2]:
ee.Initialize()

### Definir o input dos dados

In [3]:
Geom = r"C:\Users\Edson\Documents\MEGAsync\ser\ser300\proj_geoprocessamento\lago.shp"  # Poligono da área a ser analisada
DateStart = (2003, 1, 1)  # Data inicial a série temporal
DateEnd = (2003, 1, 30)  # Data final da série temporal
ResTemp = 10  # Resolução temporal da série temporal (dias para fazer o mosaico)
Escala = 500  # Escala dos cálculos
Proj = 'EPSG:4326'
SaveImg = False # (True ou False) Diz

# 2. Gera variáveis secundarias e funções para o processamento

In [5]:
Poligono = fiona.open(Geom)
PoliProj = Poligono.crs['init']
Coord = [list(x) for x in [x['geometry']['coordinates'] for x in Poligono][0][0]]
Area = ee.Geometry.Polygon(Coord, PoliProj)
region = Area.getInfo()['coordinates']  # Define o input para salvar as imagens

# Gera o DataFrame para salvar os resultados da série temporal
DateStartT = dt.datetime(DateStart[0], DateStart[1], DateStart[2])
DateEndT = dt.datetime(DateEnd[0], DateEnd[1], DateEnd[2])
index = pd.date_range(DateStartT, DateEndT, freq=str(ResTemp) + 'D')
header = ['Cobertura_Válida', 'Water_cover (km²)']
Data = pd.DataFrame( {}, index = index, columns = header)


def getqabits(image, start, end, newname):
    pattern = 0
    for i in list(range(start, end+1)):
        pattern += 2**i
        return image.select([0], [newname]).bitwiseAnd(pattern).rightShift(start)


# 3. Inicio dos cálculos

In [6]:
for date in index:  # Controla a data inicial da composição
    image = ee.Image('MODIS/006/MYD09GA/' + date.isoformat()[0:10].replace('-', '_')) # Imagem inicial da composição
    NDWI = image.expression('(Verde - NIR) / (Verde + NIR)',
                            dict(NIR=image.select('sur_refl_b02'), Verde=image.select('sur_refl_b04')))

    cloud = getqabits(image.select('state_1km'), 0, 1, 'cloud_state')
    cloud = cloud.expression("b(0) == 1 || b(0) == 2")
    pixel_valido = cloud.expression("b(0) == 0")

    agua = NDWI.expression("b(0) > 0").mask(pixel_valido)

    if ResTemp > 1:
        mosaico = pd.date_range(dt.datetime(date.year, date.month, date.day), periods=ResTemp)[1:]
        for composicao in mosaico:  # controla a intesecção dos dias no mosaico.
            image2 = ee.Image('MODIS/006/MYD09GA/' + composicao.isoformat()[0:10].replace('-', '_'))  # Imagem a ser
            # adicionada no mosaico

            NDWI2 = image2.expression('(Verde - NIR) / (Verde + NIR)',
                                      dict(NIR=image2.select('sur_refl_b02'), Verde=image2.select('sur_refl_b04')))

            cloud2 = getqabits(image2.select('state_1km'), 0, 1, 'cloud_state')
            cloud2 = cloud2.expression("b(0) == 1 || b(0) == 2")
            pixel_valido2 = cloud2.expression("b(0) == 0")

            agua2 = NDWI2.expression("b(0) > 0").mask(pixel_valido2)

            agua = ee.Image([agua.select(0), agua2.select(0)])

            pixel_valido = ee.Image([pixel_valido.select(0), pixel_valido2.select(0)])

            agua = agua.reduce(ee.Reducer.max())  # obtem o valor máximo da composicao, ou seja, se apenas uma vez foi
            # inundada
            pixel_valido = pixel_valido.reduce(ee.Reducer.max())  # obtem o máximo da composicao, ou seja, se ja ocorreu
            # alguma vez pixel válido